# Parent Company Labelling Script

This python notebook provides the code to aggregate the firm-level holdings data of institutional investors. Specifically, using hand-collected subsidiary data of the largest asset managers, an additional "Parent Company" column is added to indicate the larger investor that encompases the given subsidiary. The hand-collected data is ultimately merged to the Refinitiv investor data after the subsidiary names in both data sets are substantially simplified to maximize the chance of a successful merge. After the merge, the data should be checked for "false positives". These investors can be added to a list which will remove the parent company value for those observations.

#### Import Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
import os

#### Indicate Data Directory Locations (This Will Vary for Each User)

In [2]:
backup_dir = '00 Backup\\'
investor_data_dir = '01 HoldingsData\\'
subsidiary_data_dir = '02 Subsidiary Data\\'
false_positives_dir = '03 False Positives\\'
output_dir = '04 Output\\'

#### Dataset Information

The following code shows us the list of columns for the holdings data 2019, checks on missing values and their respective data types.

In [3]:
df_hold19 = pd.read_csv('01 HoldingsData\\2019_Data.csv')

df_hold19.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 641050 entries, 0 to 641049
Data columns (total 30 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ISIN                    641050 non-null  object 
 1   InvestorID              0 non-null       float64
 2   InvestorpermID          625189 non-null  float64
 3   Calcdate                0 non-null       float64
 4   Date                    641050 non-null  object 
 5   EarliestHoldingsDate    641050 non-null  object 
 6   HoldingsDate            641050 non-null  object 
 7   SharesHeld              641050 non-null  int64  
 8   SharesHeldValue         641050 non-null  float64
 9   PctOfSharesOutHeld      640974 non-null  float64
 10  InvParentType           641036 non-null  object 
 11  InvestorType            641036 non-null  object 
 12  InvestorTypeId          641036 non-null  float64
 13  InvestorFullName        641050 non-null  object 
 14  InvAddrCountry      

The following code shows us the list of columns for the subsidiaries data, checks on missing values and their respective data types.

In [4]:
df_subsidiaries = pd.read_csv(f'{subsidiary_data_dir}Subsidiaries_001.csv', low_memory=False)

df_subsidiaries = df_subsidiaries.drop(['Unnamed: 0'], axis = 1)
df_subsidiaries = df_subsidiaries.drop(['Unnamed: 6'], axis = 1)

df_subsidiaries['Parent Company'] = df_subsidiaries['Parent Company'].replace(['Amundi', 'Credit Agricole'], 'Credit Agricole / Amundi')

df_subsidiaries.info()

print(df_subsidiaries['Parent Company'].unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98206 entries, 0 to 98205
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Parent Company              98206 non-null  object
 1   Subsidiary Name             98206 non-null  object
 2   Source Date                 98206 non-null  int64 
 3   Percent Holdings by Parent  4507 non-null   object
 4   Source Type                 97944 non-null  object
 5   Source Link                 97836 non-null  object
dtypes: int64(1), object(5)
memory usage: 4.5+ MB
['AIG' 'Allianz' 'Credit Agricole / Amundi' 'Aviva' 'AXA Group'
 'Bank of America' 'Bank of New York Mellon' 'Barclays' 'BlackRock'
 'BNP Paribas' 'Capital Group' 'Credit Suisse' 'Deutsche Bank'
 'Fidelity Investments' 'Franklin Templeton' 'Goldman Sachs' 'HSBC'
 'ING Group' 'Invesco' 'JP Morgan Chase' 'Legal and General Group'
 'Legg Mason' 'Morgan Stanley' 'Natixis Investment Managers'
 'N

#### Constant Variables

In [5]:
##### Time Variables 
STARTYEAR = 2006
ENDYEAR = 2020
# YEARS = np.arange(STARTYEAR, ENDYEAR + 1)
### test
YEARS = [2020]

##### Data Structure Variables

COLUMN_NAMES = ['ISIN', 'InvestorID', 'InvestorpermID', 'Calcdate', 'Date',
                 'EarliestHoldingsDate', 'HoldingsDate', 'SharesHeld', 'SharesHeldValue',
                 'PctOfSharesOutHeld', 'InvParentType', 'InvestorType', 'InvestorTypeId',
                 'InvestorFullName', 'InvAddrCountry', 'InvestorAddrCity', 'FilingType',
                 'InvInvestmentStyleCode', 'InvInvmtOrientation', 'InvestorRegion',
                 'InvestorRegionId', 'month', 'day', 'year', 'HistPctOfSharesOutHeld',
                 'GROUP', 'MI', 'dup', 'TwoDig', 'IsinDummy', 'Parent Company', 'Key']

TIME_DATATYPE_COLUMNS = ['Calcdate', 'Date', 'EarliestHoldingsDate', 'HoldingsDate']

REMOVABLES = [' ', '.', ',', r"\([^()]*\)", 'aktiensgesellschaft',
              'nationalassociation', 'corporation', 'sociétéanonyme',
              'limited', 'incorporated', 'company', 'Naamlozevennootschap',
              'corp', '&co', 'sadecvsadecv', 'sadecv', 'scarl', 'sarl',
              'sàrl', 'srl', 'spa', 'sl', 'bv', 'kg', 'llp', 'zrt', 'sae',
              'sro', 'zoo', 'doo', 'isr', 'spca', 'lp', 'lda', 'llc', 'ltd',
              'plc', 'gmbh', 'mbh', 'na', 'co', 'ag', 'ab', '_nle', 'inc',
              'esop', '*', '-', '\\', '/']

### https://www.i18nqa.com/debug/utf8-debug.html
ACCENTS = {'Ã€': 'a', 'À': 'a',
           'Ã‚': 'a', 'Â': 'a',
           'Ãƒ': 'a',
           'Ã„': 'a', 'Ä': 'a',
           'Ã…': 'a', 'Å': 'a',
           'Ã‡': 'c', 'Ç': 'c',
           'Ãˆ': 'e', 'È': 'e',
           'Ã‰': 'e', 'É': 'e',
           'ÃŠ': 'e', 'Ê': 'e',
           'ÃŠ': 'e', 'Ê': 'e',
           'Ã‹': 'e', 'Ë': 'e',
           'ÃŒ': 'i', 'Ì': 'i',
           'ÃŽ': 'i', 'Î': 'i',
           'Ã‘': 'n', 'Ñ': 'n',
           'Ã’': 'o', 'Ò': 'o',
           'Ã“': 'o', 'Ó': 'o',
           'Ã”': 'o', 'Ô': 'o',
           'Ã•': 'o', 'Õ': 'o',
           'Ã–': 'o', 'Ö': 'o',
           'Ã˜': 'o', 'Ø': 'o',
           'Ã™': 'u', 'Ù': 'u',
           'Ãš': 'u', 'Ú': 'u',
           'Ã›': 'u', 'Û': 'u',
           'Ãœ': 'u', 'Ü': 'u',
           'ÃŸ': 'ss', 'ß': 'ss',
           'Ã¡': 'a', 'á': 'a',
           'Ã¢': 'a', 'â': 'a',
           'Ã£': 'a', 'ã': 'a',
           'Ã¤': 'a', 'ä': 'a',
           'Ã¥': 'a', 'å': 'a',
           'Ã¦': 'a', 'æ': 'a',
           'Ã§': 'c', 'ç': 'c',
           'Ã¨': 'e', 'è': 'e',
           'Ã©': 'e', 'é': 'e',
           'Ãª': 'e', 'ê': 'e',
           'Ã«': 'e', 'ë': 'e',
           'Ã¬': 'i', 'ì': 'i',
           'Ã®': 'i', 'î': 'i',
           'Ã¯': 'i', 'ï': 'i',
           'Ã°': 'o', 'ð': 'o',
           'Ã±': 'n', 'ñ': 'n',
           'Ã²': 'o', 'ò': 'o',
           'Ã³': 'o', 'ó': 'o',
           'Ã´': 'o', 'ô': 'o',
           'Ãµ': 'o', 'õ': 'o',
           'Ã¶': 'o', 'ö': 'o',
           'Ã¸': 'o', 'ø': 'o',
           'Ã¹': 'u', 'ù': 'u',
           'Ãº': 'u', 'ú': 'u',
           'Ã»': 'u', 'û': 'u',
           'Ã¼': 'u', 'ü': 'u',
           'Ã½': 'y', 'ý': 'y',
           'Ã¿': 'y', 'ÿ': 'y',
           'Ã ':'a', 'à': 'a',
           'Ã':'', 'Â':''
                                }


#### Holdings Data
DATATYPES = {'ISIN': 'str',
            'InvParentType': 'str',
            'InvestorType': 'str',
            'InvestorFullName': 'str',
            'InvAddrCountry': 'str',
            'InvestorAddrCity': 'str',
            'FilingType': 'str',
            'InvInvestmentStyleCode': 'str',
            'InvInvmtOrientation': 'str',
            'InvestorRegion': 'str',
            'GROUP': 'str',
            'TwoDig': 'str',}

#### Subsidiaries Data
DATATYPES_S = {'Parent Company': 'str',            
               'Subsidiary Name': 'str',
               'Source Date': 'int64',
               'Percent Holdings by Parent': 'float64',
               'Source Type': 'str',
               'Source Link': 'str'}

CHUNK_SIZE = 15000


DF_NEIGHBOR_MATRIX = pd.read_csv(f'{backup_dir}neighbor_matrix_2.csv')

INVESTORS = ['AIG', 'Allianz', 'Aviva', 'AXA Group',
             'Bank of America', 'Bank of New York Mellon', 'Barclays', 'BlackRock',
             'BNP Paribas', 'Capital Group', 'Credit Agricole / Amundi', 'Credit Suisse', 'Deutsche Bank',
             'Fidelity Investments', 'Franklin Templeton', 'Goldman Sachs', 'HSBC',
             'ING Group', 'Invesco', 'JP Morgan Chase', 'Legal and General Group',
             'Legg Mason', 'Morgan Stanley', 'Natixis Investment Managers',
             'Nippon Life Insurance', 'Northern Trust', 'Nuveen', 'Prudential',
             'State Street', 'T Rowe Price', 'UBS', 'Vanguard', 'Wellington Management', 'Wells Fargo']



FALSE_POSITIVES = []
df_false = pd.read_excel(f'{false_positives_dir}false_list.xlsx')
FALSE_POSITIVES = df_false['SubsidiaryName'].unique().tolist()



#### Functions

In [6]:
def extend_subsidiary(year, investor, df_subsidiaries = df_subsidiaries, matrix = DF_NEIGHBOR_MATRIX):

    
    relevant_year = matrix.loc[(matrix['investor'] == investor) & (matrix['year'] == year), 'relevant_year'].tolist()[0]
    
    
    if relevant_year == year:
        
        df_i_y = df_subsidiaries.loc[(df_subsidiaries['Parent Company'] == investor) & (df_subsidiaries['Source Date'] == year)]
        df_i_y = df_i_y[['Parent Company', 'Subsidiary Name']]
    
    else:
        
        df_ry = df_subsidiaries.loc[(df_subsidiaries['Parent Company'] == investor) & (df_subsidiaries['Source Date'] == relevant_year)]
        
        df_ry = df_ry[['Parent Company', 'Subsidiary Name']]
        
        df_ay = df_subsidiaries.loc[(df_subsidiaries['Parent Company'] == investor) & (df_subsidiaries['Source Date'] == year)]
        
        df_ay = df_ay[['Parent Company', 'Subsidiary Name']]
        
        df_combined = pd.concat([df_ry, df_ay])
        
        df_i_y = df_combined.drop_duplicates(subset = 'Subsidiary Name')
        
    
    return df_i_y



In [7]:
def configure_subsidiaries_years(df_s_year, removables = REMOVABLES, dtypes = DATATYPES_S):

    df_s_year['Key'] = df_s_year['Subsidiary Name']
    
    for key in ACCENTS:
        df_s_year['Key'] = df_s_year['Key'].str.replace(key, ACCENTS[key])
    
    df_s_year['Key'] = df_s_year['Key'].str.lower()

    for i in removables:
            df_s_year['Key'] = df_s_year['Key'].str.replace(i, '')
    
    
    df_s_year = df_s_year.drop_duplicates(subset = 'Key')

    return df_s_year

In [8]:
def configure_investor_data(df_i_chunk, time_dtype_columns = TIME_DATATYPE_COLUMNS, 
                            dtypes = DATATYPES, removables = REMOVABLES):
    
    
    df_i_chunk = df_i_chunk.dropna(subset = ['InvestorFullName', 'ISIN'])
    
    df_i_chunk = df_i_chunk.astype(dtypes)
    
    df_i_chunk['Key'] = df_i_chunk['InvestorFullName']
    
    for key in ACCENTS:
        df_i_chunk['Key'] = df_i_chunk['Key'].str.replace(key, ACCENTS[key])    
    
    df_i_chunk['Key'] = df_i_chunk['Key'].str.lower()
    
    df_i_chunk = df_i_chunk.drop(df_i_chunk[df_i_chunk['InvestorType'] == 'Individual Investor'].index)

    for i in removables:
        df_i_chunk['Key'] = df_i_chunk['Key'].str.replace(i, '')
    
    # for col in time_dtype_columns:
    #     df_i_chunk[col] = pd.to_datetime(df_i_chunk[col])
   

    return df_i_chunk

In [9]:
def merge_on_key(df_i_chunk, df_subsidiaries, columns = COLUMN_NAMES, false_positives = FALSE_POSITIVES):

    
    merged_chunk = pd.merge(df_i_chunk, df_subsidiaries[['Key', 'Parent Company']], on='Key', how='left')
    
    merged_chunk = merged_chunk[columns]
    
    
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('American International Group', case = False)] = 'AIG'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('Allianz', case = False)] = 'Allianz'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('Aviva ', case = False)] = 'Aviva'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('AXA ', case = False)] = 'AXA Group'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('Bank of America', case = False)] = 'Bank of America'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('Bank of New York Mellon', case = False)] = 'Bank of New York Mellon'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('BNY Mellon', case = False)] = 'Bank of New York Mellon'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('Boston Company Asset Management', case = False)] = 'Bank of New York Mellon'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('Barclays', case = False)] = 'Barclays'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('BlackRock', case = False)] = 'BlackRock'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('BNP Paribas', case = False)] = 'BNP Paribas'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('Capital World Investors', case = False)] = 'Capital Group'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('Amundi ', case = False)] = 'Credit Agricole / Amundi'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('Credit Agricole', case = False)] = 'Credit Agricole / Amundi'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('Credit Suisse', case = False)] = 'Credit Suisse'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('Deutsche Bank', case = False)] = 'Deutsche Bank'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('DWS Investments', case = False)] = 'Deutsche Bank'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('Deutsche Asset Management', case = False)] = 'Deutsche Bank'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('Fidelity', case = False)] = 'Fidelity Investments'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('Franklin Templeton', case = False)] = 'Franklin Templeton'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('Templeton', case = False)] = 'Franklin Templeton'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('Goldman Sachs', case = False)] = 'Goldman Sachs'
    # merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('Invesco', case = False)] = 'Invesco'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('JP Morgan', case = False)] = 'JP Morgan Chase'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('Legal and General', case = False)] = 'Legal and General Group'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('Legg Mason', case = False)] = 'Legg Mason'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('Morgan Stanley', case = False)] = 'Morgan Stanley'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('Natixis', case = False)] = 'Natixis Investment Managers'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('Nippon Life', case = False)] = 'Nippon Life Insurance'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('Northern Trust', case = False)] = 'Northern Trust'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('Nuveen ', case = False)] = 'Nuveen'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('Prudential ', case = False)] = 'Prudential'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('State Street', case = False)] = 'State Street'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('T Rowe Price', case = False)] = 'T Rowe Price'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('UBS ', case = False)] = 'UBS'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('Vanguard', case = False)] = 'Vanguard'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('Wellington', case = False)] = 'Wellington Management'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('Wells Fargo', case = False)] = 'Wells Fargo'
    merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('HSBC', case = False)] = 'HSBC'
    
    for i in false_positives:
        merged_chunk.loc[merged_chunk['InvestorFullName'] == i, 'Parent Company'] = ''
    
 
    return merged_chunk

#### Run Functions to Label Parent Companies

In [10]:

for year in YEARS:
    
    dfs = []
    for investor in INVESTORS:
        df_subsidiaries_i_year = extend_subsidiary(year, investor)
        dfs.append(df_subsidiaries_i_year)
    
    df_subsidiaries_year = pd.concat(dfs)
    df_subsidiaries_year = configure_subsidiaries_years(df_subsidiaries_year)
    df_subsidiaries_year.to_csv(f'{subsidiary_data_dir}{year}_subsidiary_data.csv')
    
    print(f'{year} Subsidiaries Finished')
    
    chunk_list = []
    for chunk in pd.read_csv(f'{investor_data_dir}{year}_Data.csv', chunksize = CHUNK_SIZE, encoding = 'latin1'):
        df_i_chunk = configure_investor_data(chunk)
        merged_chunk = merge_on_key(df_i_chunk, df_subsidiaries_year)
        chunk_list.append(merged_chunk)
        
    df_merged = pd.concat(chunk_list)
    df_merged.to_csv(f'{output_dir}{year}_InvestorData.csv', encoding = 'latin1')
    
    print(f'{year} Merge Finished')

C:\Users\DERMINA1\AppData\Local\Temp/ipykernel_29056/1928241043.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_s_year['Key'] = df_s_year['Key'].str.replace(i, '')
C:\Users\DERMINA1\AppData\Local\Temp/ipykernel_29056/1928241043.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df_s_year['Key'] = df_s_year['Key'].str.replace(i, '')


2020 Subsidiaries Finished


C:\Users\DERMINA1\AppData\Local\Temp/ipykernel_29056/217490302.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_i_chunk['Key'] = df_i_chunk['Key'].str.replace(i, '')
C:\Users\DERMINA1\AppData\Local\Temp/ipykernel_29056/217490302.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
  df_i_chunk['Key'] = df_i_chunk['Key'].str.replace(i, '')
C:\Users\DERMINA1\AppData\Local\Temp/ipykernel_29056/287290384.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_chunk['Parent Company'][merged_chunk['InvestorFullName'].str.contains('Allianz', case = False)] = 'Allianz'
C:\Users\DERMINA1\AppData\Lo

2020 Merge Finished


#### Export Matching Files for each year

In [11]:
for year in YEARS:
    
    df_year = pd.read_csv(f'{output_dir}{year}_InvestorData.csv', encoding = 'latin')
    df_year = df_year[df_year['Parent Company'].notna()]
    df_year = df_year.drop_duplicates(subset = ['InvestorFullName', 'Parent Company'])
    df_year.to_excel(f'{false_positives_dir}{year}_Matchings.xlsx', encoding = 'latin1')
    
    print(f'{year} unique matchings finished')





2020 unique matchings finished


In [12]:
print('Finished')

Finished
